# GBC Replication

The following notebook will use the PDP to analyse the results of a recent paper *citation here*, to do this the gradient boosting classifyer that comes stock in scikit-learn will be used as it comes with PDP functions. 

In [32]:
import pandas as pd
import numpy as np
import pandas as pd
import numpy as np
from sklearn import cross_validation, metrics   #Additional scklearn functions
import matplotlib.pyplot as plt
%matplotlib inline
trainData = pd.read_csv('1e7data_train.csv', index_col=0)
testData = pd.read_csv('1e7data_holdout.csv', index_col=0)
df = trainData.astype('float64')
dftest = testData.astype('float64')

In [60]:
from sklearn.preprocessing import normalize, scale

features = []
features += ['daOverRH1', 'daOverRH2']
features += ['mindaOverRH', 'maxdaOverRH']
features += ['norm_std_a1', 'norm_max_a1', 'norm_std_window10_a1', 'norm_max_window10_a1']
features += ['norm_std_a2', 'norm_max_a2', 'norm_std_window10_a2', 'norm_max_window10_a2']
features += ['norm_std_a3', 'norm_max_a3', 'norm_std_window10_a3', 'norm_max_window10_a3']
features += ['avg_ecross1', 'std_ecross1', 'max_ecross1', 'min_ecross1']
features += ['avg_ecross2', 'std_ecross2', 'max_ecross2', 'min_ecross2']
features += ['avg_ecross3', 'std_ecross3', 'max_ecross3', 'min_ecross3']
features += ['norm_a1_slope', 'norm_a2_slope', 'norm_a3_slope']
features += ['norm_LyapunovTime']

X_train = scale(df[features].astype('float64'))
y_train = df['Stable'].astype('float64')

X_test = dftest[features]

# print np.isnan(X_train.any())
# print np.isfinite(y_train.all())
# print np.isnan(X_test.any())

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [61]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
import xgboost as xgb
from xgboost.sklearn import XGBClassifier

#  the best XGBoost model parameters:

# model0 = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1.0,
#          gamma=0, learning_rate=0.002, max_delta_step=0, max_depth=8,
# min_child_weight=1.2, missing=None, n_estimators=5000, nthread=-1,
#         objective='binary:logistic', reg_alpha=0, reg_lambda=1,
#         scale_pos_weight=1, seed=27, silent=True, subsample=0.5)


# True
model = GradientBoostingClassifier(loss='deviance', learning_rate=0.002, n_estimators=5000,
                                   subsample=.5, min_samples_split=2, 
                                   min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=8,
                                   init=None, random_state=42, 
                                   max_leaf_nodes=8, warm_start=True, max_features=.5)

model.fit(X_train, y_train)

dtest_predictions = model.predict(X_test) 
dtest_predprob = model.predict_proba(X_test)

#Print model report:
print "\nModel Report"
print "Accuracy : %.4g" % metrics.accuracy_score(dftest['Stable'].values, dtest_predictions)
print "AUC Score (Test): %f" % metrics.average_precision_score(dftest['Stable'], dtest_predprob)
print 'ROC_AUC_Score: ', metrics.roc_auc_score(dftest['Stable'], dtest_predprob)


ValueError: Input contains NaN, infinity or a value too large for dtype('float32').

found online: 

 max_features=None is like column_subsample
 gbm ------ xgboost
 n.trees ------> nrounds
 interaction.depth ------> max.depth
 bag.fraction ------> subsample
 distribution ------> objective
 n.cores ------> nthread
 n.minobsinnode ------> min_child_weight?




 Notes:

Is gamma like min_impurity_split? :
    
    gamma [default=0, alias: min_split_loss]
minimum loss reduction required to make a further partition on a leaf node of the tree. The larger, the more conservative the algorithm will be.


    min_impurity_split : float, optional (default=1e-7)
Threshold for early stopping in tree growth. A node will split if its impurity is above the threshold, otherwise it is a leaf.

Max nodes like max depth? 
    max_depth [default=6]
maximum depth of a tree, increase this value will make the model more complex / likely to be overfitting.
range: [1,∞]

    max_leaf_nodes : int or None, optional (default=None)
Grow trees with max_leaf_nodes in best-first fashion. Best nodes are defined as relative reduction in impurity. If None then unlimited number of leaf nodes. 

IE is inf = none? 

Some notes on the two algorithms: 

In order to evaluate the split point, you would have to compute L(y,Hi−1+hi)L(y,Hi−1+hi) where LL is the cost function, yy the target, Hi−1Hi−1 the model built until now, and hihi the current addition. Notice that this is not what XGBoost is doing; they are simplifying the cost function LL by a Taylor Expansion, which leads to a very simple function to compute. They have to compute the Gradient and the Hessian of LL with respect to Hi−1Hi−1, and they can reuse those number for all potential splits at stage ii, making the overral computation fast. You can check Loss function Approximation With Taylor Expansion (CrossValidated Q/A) for more details, or the derivation in their paper.

The point is that they have found a way to approximate L(y,Hi−1+hi)L(y,Hi−1+hi) efficiently. If you were to evaluate LL fully, without insider knowledge allowing optimisation or avoidance or redundant computation, it would take more time per split. It this regard, it is an approximation. However, other gradient boosting implementations also use a proxy cost functions to evaluate the splits, and I do not know whether XGBoost approximation is quicker in this regards than the others.

http://stats.stackexchange.com/questions/202858/loss-function-approximation-with-taylor-expansion/204377

Some notes on max features as a modulator: 

max_features : int, float, string or None, optional (default=None)
The number of features to consider when looking for the best split:
If int, then consider max_features features at each split.
If float, then max_features is a percentage and int(max_features * 
n_features) features are considered at each split.
If “auto”, then max_features=sqrt(n_features).
If “sqrt”, then max_features=sqrt(n_features).
If “log2”, then max_features=log2(n_features).
If None, then max_features=n_features.

Choosing max_features < n_features leads to a reduction of variance and 
an increase in bias.

In [21]:
# False

model2 = GradientBoostingClassifier(loss='deviance', learning_rate=0.002, n_estimators=5000,
                                   subsample=.5, min_samples_split=2, 
                                   min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_depth=8,
                                   init=None, random_state=42, 
                                   max_leaf_nodes=8, warm_start=False, max_features=.5)

In [22]:
model_train = model2.fit(X_train, y_train)
dtest_predictions = model2.predict(X_test) 
dtest_predprob = model2.predict_proba(X_test)

#Print model report:
print "\nModel Report"
print "Accuracy : %.4g" % metrics.accuracy_score(dftest['Stable'].values, dtest_predictions)
print "AUC Score (Test): %f" % metrics.average_precision_score(dftest['Stable'], dtest_predprob)
print 'ROC_AUC_Score: ', metrics.roc_auc_score(dftest['Stable'], dtest_predprob)

ValueError: Input contains NaN, infinity or a value too large for dtype('float32').